# Unity Catalog Data Advisor Agent - Test Notebook

This notebook tests the Unity Catalog Data Advisor Agent.

## Setup
1. Upload all project files to your Databricks workspace
2. Make sure `config.py` has your `LLM_ENDPOINT_NAME` and `VECTOR_SEARCH_INDEX_NAME` configured
3. Run the cells below to test the agent

In [0]:
%pip install -U -qqqq backoff databricks-openai uv databricks-agents mlflow-skinny[databricks]
dbutils.library.restartPython()

In [0]:
dbutils.library.restartPython()

In [0]:

# Import the agent
from agent import AGENT
from mlflow.types.responses import ResponsesAgentRequest

print("Agent loaded successfully!")
print(f"LLM Endpoint: {AGENT.llm_endpoint}")
print(f"Available tools: {list(AGENT._tools_dict.keys())}")

## Test 1: Basic Query

In [0]:
# Test basic query
request = ResponsesAgentRequest(
    input=[{"role": "user", "content": "What tables contain Peloton data?"}],
    custom_inputs={"session_id": "test-session-1"}
)

print("=" * 80)
print()

response = AGENT.predict(request)

# Print the response in a readable format
for output in response.output:
    if output.type == "text":
        print(output.content)
    elif output.type == "function_call_output":
        print(f"\n[Tool Output]: {output.content}")

## Test 2: Streaming Query

In [0]:
# Test streaming query
request = ResponsesAgentRequest(
    input=[{"role": "user", "content": "What tables contain Peloton data?"}],
    custom_inputs={"session_id": "test-session-2"}
)

print("\nStreaming response chunks:")
for chunk in AGENT.predict_stream(request):
    chunk_data = chunk.model_dump(exclude_none=True)
    if chunk_data.get("type") == "response.output_item.done":
        item = chunk_data.get("item", {})
        if item.get("type") == "text":
            print(item.get("content", ""), end="", flush=True)
        elif item.get("type") == "function_call_output":
            print(f"\n[Tool Output]: {item.get('content', '')}")

## Test 3: Custom Query

In [0]:
# Test with your own query
request = ResponsesAgentRequest(
    input=[{"role": "user", "content": "Your question here"}],
    custom_inputs={"session_id": "test-session-3"}
)

response = AGENT.predict(request)

for output in response.output:
    if output.type == "text":
        print(output.content)
    elif output.type == "function_call_output":
        print(f"\n[Tool Output]: {output.content}")